In [1]:
from mxnet import gluon,init,nd
from mxnet.gluon import data as gdata,nn
import os,sys

# 1. 获取数据 / 生成iter
---

In [7]:
mnist_path = r"C:\Users\59352\.mxnet\datasets\fashion-mnist"
def load_data_fashion_mnist(batch_size,resize = None):
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)
    
    mnist_train = gdata.vision.FashionMNIST(root=mnist_path,train=True)
    mnist_test = gdata.vision.FashionMNIST(root=mnist_path,train=False)
    
    num_workers = 0 if sys.platform.startswith('win') else 4
    
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size=batch_size,shuffle=True,num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                batch_size=batch_size,shuffle=False,num_workers=num_workers)
    return train_iter,test_iter

In [ ]:
batch_size = 128
train_iter,test_iter = load_data_fashion_mnist(batch_size,resize=224)

# 2. 生成模型
---
> input: (1,1,224,224)

- network construction:(all activation are `relu`)
    ```
    1. conv 1 output : ( 1 , 96 , 54 , 54 )
    2. pool 1 output : ( 1 , 96 , 26 , 26 )
    3. conv 2 output : ( 1 , 256 , 26 , 26 )
    4. pool 2 output : ( 1 , 256 , 12 , 12 )
    5. conv 3 output : ( 1 , 384 , 12 , 12 )
    6. conv 4 output : ( 1 , 384 , 12 , 12 )....??????干饭层?????
    7. pool 3 output : ( 1 , 256 , 5 , 5 )
    ```
    
    ```
    8. dense 1 output : ( 1 , 4096 )
    9. dropout 1 output : ( 1 , 4096 )
    10. dense 2 output : ( 1,4096 )
    11. dropout 2 output : ( 1 , 4096 )
    12. dense 3 output : ( 1 , 10 )
    ```

In [ ]:
net = nn.Sequential()
net.add(nn.Conv2D(channels=96,kernel_size=11,strides=4,activation='relu'),
       nn.MaxPool2D)

# 3. 训练
---

In [ ]:
lr,num_epochs,ctx = 0.01,5,d2l.try_gpu()
